In [0]:
from pyspark.sql.functions import isnan, when, count, col, to_date, year, hour, minute, to_timestamp, lit, concat, expr, regexp_replace
from pyspark.sql.types import *

In [0]:
df = spark.read.format('csv').options(delimiter=';', header='true', infer_schema='true') \
        .load('/Volumes/mateus_dev/default/files/acidentes_brasil.csv')

In [0]:
df.show(5)

+------+------------+------------+--------+---+---+-----+--------------------+--------------------+--------------------+----------------------+---------+-----------+----------------------+----------+-----------+--------+-------+------+-------------+--------------+------+---------+-------+--------+------------+------------+--------+---------+--------------+
|    id|data_inversa|  dia_semana| horario| uf| br|   km|           municipio|      causa_acidente|       tipo_acidente|classificacao_acidente| fase_dia|sentido_via|condicao_metereologica|tipo_pista|tracado_via|uso_solo|pessoas|mortos|feridos_leves|feridos_graves|ilesos|ignorados|feridos|veiculos|    latitude|   longitude|regional|delegacia|           uop|
+------+------------+------------+--------+---+---+-----+--------------------+--------------------+--------------------+----------------------+---------+-----------+----------------------+----------+-----------+--------+-------+------+-------------+--------------+------+---------+-

In [0]:
df.select(df.horario).show()

+--------+
| horario|
+--------+
|05:40:00|
|06:00:00|
|06:00:00|
|10:08:00|
|12:10:00|
|14:15:00|
|14:25:00|
|14:50:00|
|16:10:00|
|15:55:00|
|19:30:00|
|19:20:00|
|18:00:00|
|18:10:00|
|03:10:00|
|02:00:00|
|04:15:00|
|07:00:00|
|05:30:00|
|06:10:00|
+--------+
only showing top 20 rows



In [0]:
df = df.withColumn("data_inversa", to_date(col("data_inversa"), "dd/MM/yyyy"))
df = df.withColumn("ano", year(col("data_inversa")))

In [0]:
df.select(df.horario).show()

+--------+
| horario|
+--------+
|05:40:00|
|06:00:00|
|06:00:00|
|10:08:00|
|12:10:00|
|14:15:00|
|14:25:00|
|14:50:00|
|16:10:00|
|15:55:00|
|19:30:00|
|19:20:00|
|18:00:00|
|18:10:00|
|03:10:00|
|02:00:00|
|04:15:00|
|07:00:00|
|05:30:00|
|06:10:00|
+--------+
only showing top 20 rows



In [0]:
output_path = "/Volumes/mateus_dev/default/files/car_accidents"

df.write.mode("overwrite") \
    .partitionBy("ano", "UF") \
    .parquet(output_path)

In [0]:
df_parquet = spark.read.parquet("/Volumes/mateus_dev/default/files/car_accidents")



['id',
 'data_inversa',
 'dia_semana',
 'horario',
 'br',
 'km',
 'municipio',
 'causa_acidente',
 'tipo_acidente',
 'classificacao_acidente',
 'fase_dia',
 'sentido_via',
 'condicao_metereologica',
 'tipo_pista',
 'tracado_via',
 'uso_solo',
 'pessoas',
 'mortos',
 'feridos_leves',
 'feridos_graves',
 'ilesos',
 'ignorados',
 'feridos',
 'veiculos',
 'latitude',
 'longitude',
 'regional',
 'delegacia',
 'uop',
 'ano',
 'UF']

In [0]:
for column in df_parquet.columns:
    df_parquet = df_parquet.filter(col(column).isNotNull())

In [0]:
df_parquet = df_parquet.filter(df_parquet.tipo_acidente.isNotNull())

In [0]:
df_parquet.toJSON()

---------------------------------------------------------------------------
PySparkAttributeError                     Traceback (most recent call last)
File <command-700950083878940>, line 1
----> 1 df_parquet.toJSON()

File /databricks/python/lib/python3.10/site-packages/pyspark/sql/connect/dataframe.py:1716, in DataFrame.__getattr__(self, name)
   1714 def __getattr__(self, name: str) -> "Column":
   1715     if name in ["_jseq", "_jdf", "_jmap", "_jcols", "rdd", "toJSON"]:
-> 1716         raise PySparkAttributeError(
   1717             error_class="JVM_ATTRIBUTE_NOT_SUPPORTED", message_parameters={"attr_name": name}
   1718         )
   1719     elif name in [
   1720         "checkpoint",
   1721         "localCheckpoint",
   1722     ]:
   1723         raise PySparkNotImplementedError(
   1724             error_class="NOT_IMPLEMENTED",
   1725             message_parameters={"feature": f"{name}()"},
   1726         )

PySparkAttributeError: [JVM_ATTRIBUTE_NOT_SUPPORTED] Directly 

In [0]:
df_parquet.filter(df_parquet.tipo_acidente.isNull()).count()

0

In [0]:
df_parquet.select(df_parquet.horario).show()

+--------+
| horario|
+--------+
|19:10:00|
|18:30:00|
|18:30:00|
|16:40:00|
|18:00:00|
|17:30:00|
|15:00:00|
|09:35:00|
|14:50:00|
|13:15:00|
|07:40:00|
|21:40:00|
|20:00:00|
|16:10:00|
|13:45:00|
|15:50:00|
|11:00:00|
|14:30:00|
|19:10:00|
|06:00:00|
+--------+
only showing top 20 rows



In [0]:
df_parquet = df_parquet.withColumn("id",df_parquet.id.cast(IntegerType()))
df_parquet = df_parquet.withColumn("data_inversa", to_date(col("data_inversa"), "dd/MM/yyyy"))
df_parquet = df_parquet.withColumn("data_horario", to_timestamp(concat(col("data_inversa"), lit(" "), col("horario")), "yyyy-MM-dd HH:mm:ss"))
df_parquet = df_parquet.withColumn("br",df_parquet.br.cast(IntegerType()))
df_parquet = df_parquet.withColumn("km",df_parquet.km.cast(FloatType()))
df_parquet = df_parquet.withColumn("pessoas",df_parquet.pessoas.cast(IntegerType()))
df_parquet = df_parquet.withColumn("mortos",df_parquet.mortos.cast(IntegerType()))
df_parquet = df_parquet.withColumn("feridos_leves",df_parquet.feridos_leves.cast(IntegerType()))
df_parquet = df_parquet.withColumn("feridos_graves",df_parquet.feridos_graves.cast(IntegerType()))
df_parquet = df_parquet.withColumn("ilesos",df_parquet.ilesos.cast(IntegerType()))
df_parquet = df_parquet.withColumn("ignorados",df_parquet.ignorados.cast(IntegerType()))
df_parquet = df_parquet.withColumn("feridos",df_parquet.feridos.cast(IntegerType()))
df_parquet = df_parquet.withColumn("veiculos",df_parquet.veiculos.cast(IntegerType()))
df_parquet = df_parquet.withColumn("latitude", regexp_replace(col("latitude"), ",", ".").cast(DoubleType()))
df_parquet = df_parquet.withColumn("longitude", regexp_replace(col("longitude"), ",", ".").cast(DoubleType()))
df_parquet = df_parquet.withColumn("ano",df_parquet.ano.cast(IntegerType()))

df_parquet.printSchema()


root
 |-- id: integer (nullable = true)
 |-- data_inversa: date (nullable = true)
 |-- dia_semana: string (nullable = true)
 |-- horario: string (nullable = true)
 |-- br: integer (nullable = true)
 |-- km: float (nullable = true)
 |-- municipio: string (nullable = true)
 |-- causa_acidente: string (nullable = true)
 |-- tipo_acidente: string (nullable = true)
 |-- classificacao_acidente: string (nullable = true)
 |-- fase_dia: string (nullable = true)
 |-- sentido_via: string (nullable = true)
 |-- condicao_metereologica: string (nullable = true)
 |-- tipo_pista: string (nullable = true)
 |-- tracado_via: string (nullable = true)
 |-- uso_solo: string (nullable = true)
 |-- pessoas: integer (nullable = true)
 |-- mortos: integer (nullable = true)
 |-- feridos_leves: integer (nullable = true)
 |-- feridos_graves: integer (nullable = true)
 |-- ilesos: integer (nullable = true)
 |-- ignorados: integer (nullable = true)
 |-- feridos: integer (nullable = true)
 |-- veiculos: integer (null

In [0]:
df_parquet.select(df_parquet.data_horario).show(5)

+-------------------+
|       data_horario|
+-------------------+
|2020-06-05 19:10:00|
|2020-06-05 18:30:00|
|2020-06-05 18:30:00|
|2020-06-05 16:40:00|
|2020-06-05 18:00:00|
+-------------------+
only showing top 5 rows



In [0]:
df_parquet.show(5)

+------+------------+-----------+--------+---+-----+---------+--------------------+--------------------+----------------------+-----------+-----------+----------------------+----------+-----------+--------+-------+------+-------------+--------------+------+---------+-------+--------+------------+------------+--------+---------+--------------+----+---+-------------------+
|    id|data_inversa| dia_semana| horario| br|   km|municipio|      causa_acidente|       tipo_acidente|classificacao_acidente|   fase_dia|sentido_via|condicao_metereologica|tipo_pista|tracado_via|uso_solo|pessoas|mortos|feridos_leves|feridos_graves|ilesos|ignorados|feridos|veiculos|    latitude|   longitude|regional|delegacia|           uop| ano| UF|       data_horario|
+------+------------+-----------+--------+---+-----+---------+--------------------+--------------------+----------------------+-----------+-----------+----------------------+----------+-----------+--------+-------+------+-------------+--------------+--

In [0]:
df_parquet = df_parquet.withColumn("percentual_fatalidades", (df_parquet.mortos/df_parquet.pessoas) * 100)

In [0]:
df_parquet.filter(df_parquet.classificacao_acidente == "NA").count()

4

In [0]:
df_parquet = df_parquet.withColumn("classificacao_acidente", when(col("classificacao_acidente") == "NA", 
         when(col("mortos") > 0, "Com Vitimas Fatais")
        .when((col("feridos_leves") + col("feridos_graves")) > 0, "Com Vitimas Feridas")
        .otherwise("Sem Vitimas"))
    .otherwise(col("classificacao_acidente")))

In [0]:
df_parquet.select(df_parquet.classificacao_acidente).distinct().show()

+----------------------+
|classificacao_acidente|
+----------------------+
|    Com Vitimas Fatais|
|   Com Vitimas Feridas|
|           Sem Vitimas|
+----------------------+



In [0]:
df.write.mode("overwrite").partitionBy("ano").option("compression", "snappy").parquet("/Volumes/mateus_dev/default/files/silver/car_accidents")